#### 使用FP-growth算法来高效发现频繁项集
- FP-growth将数据集存储在一个特定的称作FP（Frequent Pattern）树的结构之后发现频繁项集或者频繁项对，即常在一块出现的元素项的集合FP树。
- FP-growth算法只需要对数据库进行两次扫描，而Apriori算法对于每个潜在的频繁项集都会扫描数据集判定给定模式是否频繁，因此FP-growth算法的速度要比Apriori快。
- FP-growth算法的基本过程为：（1）构建FP树；（2）从FP树中挖掘频繁项集。
- 优点：一般快于Apriori； 缺点：实现比较困难，在某些数据集上性能会下降。


In [2]:
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None    #nodeLink用于链接相似的元素项
        self.parent = parentNode    #指向当前节点的父节点
        self.children = {}
        
    def inc(self, numOccur):
        '''
        对count变量增加指定值
        '''
        self.count += numOccur
        
    def disp(self, ind =1):
        '''
        将树以文本形式显示
        '''
        print(' '*ind, self.name, ' ',self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [3]:
rootNode = treeNode('pyramid',9,None)
rootNode.children['eye']=treeNode('eye',13,None)

In [4]:
rootNode.disp()

  pyramid   9
   eye   13


In [5]:
rootNode.children['phoenix'] = treeNode('phoenix',3,None)
rootNode.disp()

  pyramid   9
   eye   13
   phoenix   3


In [6]:
def createTree(dataSet, minSup =1):
    '''
    使用数据集以及最小支持度作为参数构建FP树
    '''
    headerTable = {}
    for trans in dataSet:   #第一次遍历数据集
        for item in trans:    #统计每一个元素项出现的额度
            headerTable[item] = headerTable.get(item,0) + dataSet[trans]
    for k in list(headerTable.keys()):    #移除不满足最小支持度的元素项
        if headerTable[k] < minSup:
            del(headerTable[k]) 
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0: return None, None    #如果没有元素项满足要求，就退出
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():    #第二次遍历数据集，根据全局频率对每个事物中的元素进行排序
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD)>0:    #使用排序后的频率项集对树进行填充
            orderedItems = [v[0] for v in sorted(localD.items(), key = lambda p: p[1], reverse = True)]
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable

def updateTree(items, inTree, headerTable, count):
    '''
    使FP树生长
    '''
    if items[0] in inTree.children:    #测试事务中的第一个元素项是否作为子节点存在
        inTree.children[items[0]].inc(count)    #若存在更新该元素项的计数
    else:   #不存在，创建一个新的treeNode并将其作为一个子节点添加到树中
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #头指针表也要更新以指向新的节点
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:    #不断迭代调用自身，每次调用时会去掉列表中的第一个元素
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)
        
def updateHeader(nodeToTest, targetNode):   
    '''
    确保节点链接指向树中该元素项的每一个实例
    '''
    while (nodeToTest.nodeLink != None):    
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [7]:
def loadSimpDat():
    '''
    返回一个事务列表
    '''
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

def createInitSet(dataSet):
    '''
    实现从列表到字典的类型转换过程
    '''
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [8]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [9]:
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [10]:
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()

  Null Set   1
   z   5
    r   1
    x   3
     y   3
      t   1
       s   1
      r   1
       t   1
      s   1
       t   1
   x   1
    r   1
     s   1


#### 从一棵FP树中挖掘频繁项集的三个步骤：
- step1：从FP树中获得条件模式基；
- step2：利用条件模式基，构建一个条件FP树；
- step3：迭代重复步骤（1）和步骤（2），直到树包含一个元素项为止。
- 对于每一个元素项，获得其对应的条件模式基（conditional pattern base）。条件模式基是以所查元素项为结尾的路径集合。每一条路径其实都是一条前缀路径。简而言之，一条前缀路径是介于所查找元素项与树根节点之间的所有内容。

In [11]:
def ascendTree(leafNode, prefixPath): 
    '''
    上溯FP树，收集遇到的所有元素项的名称
    '''
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)
    
def findPrefixPath(basePat, treeNode): 
    '''
    为给定元素项生成一个条件模式基
    '''
    condPats = {}    #条件模式基字典
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1: 
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [12]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [13]:
findPrefixPath('z', myHeaderTab['z'][1])

{}

In [14]:
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'z'}): 1, frozenset({'x'}): 1, frozenset({'x', 'y', 'z'}): 1}

In [15]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    '''
    递归查找频繁项集
    '''
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[0])]   #默认从小到大排序
    for basePat in bigL:   #从头指针表的底端开始
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)  #将每一个频繁项添加到频繁项集列表
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        myCondTree, myHead = createTree(condPattBases, minSup)
        if myHead != None: #如果树中又元素项的话，调用mineTree()
            print('conditional tree for: ',newFreqSet)
            myCondTree.disp(1)            
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [16]:
freqItems =[]

In [17]:
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

conditional tree for:  {'s'}
  Null Set   1
   x   3
conditional tree for:  {'t'}
  Null Set   1
   y   3
    x   3
     z   3
conditional tree for:  {'t', 'x'}
  Null Set   1
   y   3
conditional tree for:  {'t', 'z'}
  Null Set   1
   x   3
    y   3
conditional tree for:  {'t', 'z', 'y'}
  Null Set   1
   x   3
conditional tree for:  {'x'}
  Null Set   1
   z   3
conditional tree for:  {'y'}
  Null Set   1
   x   3
    z   3
conditional tree for:  {'y', 'z'}
  Null Set   1
   x   3


#### 示例：从新闻网站点击流中挖掘

In [18]:
parseDat = [line.split() for line in open('data/kosarak.dat').readlines()]

In [19]:
initSet = createInitSet(parseDat)
myFPtree, myHeaderTab = createTree(initSet, 100000)
myFPtree.disp()

  Null Set   1
   3   76514
    1   12917
   1   16829
   6   412762
    11   261773
     3   117401
      1   34141
     1   43366
    3   68888
     1   13436
    1   16461
   11   21190
    3   9718
     1   1565
    1   1882


In [20]:
myFreqList = []
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)

conditional tree for:  {'1'}
  Null Set   1
   6   107404
conditional tree for:  {'11'}
  Null Set   1
   6   261773
conditional tree for:  {'3'}
  Null Set   1
   6   186289
    11   117401
   11   9718
conditional tree for:  {'11', '3'}
  Null Set   1
   6   117401


In [21]:
len(myFreqList)

9

In [22]:
myFreqList

[{'1'},
 {'1', '6'},
 {'11'},
 {'11', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'6'}]